In [7]:
%load_ext autoreload
%autoreload 2
import yaml
from pathlib import Path
import sys
import os
import pprint
current_dir = Path(os.getcwd()).resolve()
if current_dir.name == "LLMPolReasonEval": # uruchomione w Jupyter Lab
    project_root = current_dir
else:  # uruchomione w PyCharm
    project_root = current_dir.parents[2]
print(f"Project root: {project_root}")
sys.path.append(str(project_root / "src"))

from llm_pol_reason_eval.qa_engine.llm_qa_engine import LLMQAEngine
from llm_pol_reason_eval.qa_engine.inference_client import HuggingFaceClient

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Project root: C:\Users\piotr\PycharmProjects\LLMPolReasonEval


In [8]:
EXPERIMENT_NAME = "qwen-thinking-matura"

# Wczytaj pliki konfiguracyjne
RUN_CONFIG_FILE = "config/runs/answer_generation_run.yaml"
MODELS_CONFIG_FILE = "config/models.yaml"

with open(project_root / RUN_CONFIG_FILE, 'r', encoding='utf-8') as f:
    run_config = yaml.safe_load(f)['experiments'][EXPERIMENT_NAME]

with open(project_root / MODELS_CONFIG_FILE, 'r', encoding='utf-8') as f:
    models_config = yaml.safe_load(f)

print(f"Uruchamiam eksperyment: {run_config.get('task_name')}")

Uruchamiam eksperyment: Qwen3 1.7B - Odpowiedzi z myśleniem 'krok po kroku'


In [9]:
model_key = run_config['model']
model_cfg = models_config[model_key]

run_overrides = run_config.get("param_overrides", {})
final_gen_params = model_cfg['generation_params'].copy()
final_gen_params.update(run_overrides.get('default', {}))

# Inicjalizacja klienta inferencji - to może potrwać chwilę
inference_client = HuggingFaceClient(
    model_path=model_cfg['path'],
    default_generation_params=final_gen_params
)

engine = LLMQAEngine(
    model_name=model_key,
    model_path=model_cfg['path'],
    inference_client=inference_client
)

HuggingFaceClient: Inicjalizacja modelu Qwen/Qwen3-1.7B na urządzeniu: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

HuggingFaceClient: Domyślna konfiguracja generowania: GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "max_new_tokens": 768,
  "pad_token_id": 151643,
  "temperature": 0.6,
  "top_k": 20,
  "top_p": 0.95
}



In [11]:
input_dataset_path = project_root / run_config['input_dataset']

# Użyjemy query, żeby nie przetwarzać całego datasetu podczas testów w notatniku
# Możesz usunąć `query`, aby przetworzyć cały plik
target_question_ids = ["MPOP-P1-100-A-2405_zadanie_14", "EPOP-P1-100-2305_zad_1"]
query = lambda q: q.get("question_id") in target_question_ids

per_type_params = run_config.get("param_overrides", {}).get('per_type')

results = engine.generate_answers(
    dataset_filepath=str(input_dataset_path),
    param_overrides=run_overrides,
    query=query
)


--- Przetwarzanie batcha | Typ: closed_MTF, Pytania: 1 ---


TemplateNotFound: default\system.jinja2

In [ ]:
print("--- WYGENEROWANE ODPOWIEDZI ---")
pprint.pprint(results)

# Zapisz wyniki
output_dir = project_root / run_config['output_dir']
output_path = output_dir / f"answers_notebook_{EXPERIMENT_NAME}.json"
engine.save_results(output_filepath=str(output_path))